In [ ]:
import datetime
import json
import difflib

In [ ]:
ld = {}
with open('/Users/Mr_QL/Desktop/local_result.json', 'r', encoding='utf-8') as f:
    ld = json.load(f)
    print(ld.keys())

with open('/Users/Mr_QL/Desktop/scan_result.json', 'r', encoding='utf-8') as f:
    scan_data = json.load(f)
    print(ld.keys())

In [ ]:
def scan_differences(scan_data, local_data): 
    diff_add = {}
    diff_del= {}
    for key in scan_data:
        if key in local_data:
            if isinstance(scan_data[key], dict) and isinstance(local_data[key], dict):
                # 当两个字典的value仍为字典时，递归比较嵌套字典的差异
                nested_add, nested_del = scan_differences(scan_data[key], local_data[key])
                if nested_add:
                    diff_add[key] = nested_add
                if nested_del:
                    diff_del[key] = nested_del

            elif scan_data[key] != local_data[key]:
                # 当字典同一个key的value不同时,记录差异内容
                diff_add[key] = scan_data[key]
                diff_del[key] = local_data[key]
        else:
            #扫描数据中发现有新增的内容，本地没有
            diff_add[key] = scan_data[key]

    for key in local_data:
        if key not in scan_data:
            #扫描数据中缺失了某些本地数据中的内容
           diff_del[key] = local_data[key]
    return diff_add ,diff_del
                

In [ ]:
def diff_str(scan_data, local_data):
    d = difflib.Differ()
    diff = list(d.compare(scan_data.splitlines(), local_data.splitlines()))
    diff_add, diff_del = [], []
    for line in diff:
        if line.startswith(' '):            #一致内容
            continue
        elif line.startswith('- '):         #包含在第一个系列行中，但不包含第二个
            diff_add.append(line[2:])
        elif line.startswith('+ '):         #包含在第二个系列行中，但不包含第一个
            diff_del.append(line[2:])
    return ", ".join(diff_add), ", ".join(diff_del)

In [ ]:
d = { }
for fkey in scan_data.keys():
    d[fkey] = {}
    if(isinstance(scan_data[fkey],dict)):
        for skey in scan_data[fkey].keys():
            if(isinstance(scan_data[fkey][skey], dict)):
                _add, _del = scan_differences(scan_data[fkey][skey], ld[fkey][skey])
                d[fkey]["add"+skey] = _add
                d[fkey]["del"+skey] = _del
            elif(scan_data[fkey][skey] != ld[fkey][skey]):
                if(isinstance(scan_data[fkey][skey], str)):
                    d[fkey]["add"+skey], d[fkey]["del"+skey] = diff_str(scan_data[fkey][skey], ld[fkey][skey])
                else:
                    d[fkey]["add"+skey] = scan_data[fkey][skey]
                    d[fkey]["del"+skey] =  ld[fkey][skey]
                    
            else:
                d[fkey]["add"+skey] = {}
                d[fkey]["del"+skey] = {}